In [8]:
#Import necessary libraries
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
import socket
from paho.mqtt import client as mqtt_client
import random

In [9]:
UDP_IP = "0.0.0.0"
UDP_PORT = 1234

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))

In [10]:
broker = 'broker.hivemq.com'
port = 1883
topic = "IOTProj1"
client_id = f'python-mqtt-{random.randint(0, 1000)}'
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client
    
client = connect_mqtt()
client.loop_start()

In [11]:
#Function to split dataset necesary steps
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1:
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 


In [12]:
#Data preprocessing
df=pd.read_csv('consumption.csv')
df_consume=df['Global_active_power']
df["Global_active_power"] = df['Global_active_power'].str.replace('?', '0')
df_consume.fillna(value=0)
df_consume_day=df_consume[0:1442].astype(float)
#df_kwh=df_consume_day.cumsum()
#df_kwh=df_kwh*(1/60)
#df_kwh
df_consume_day

Connected to MQTT Broker!


0       2.580
1       2.552
2       2.550
3       2.550
4       2.554
        ...  
1437    0.464
1438    0.462
1439    0.460
1440    0.442
1441    0.370
Name: Global_active_power, Length: 1442, dtype: float64

In [13]:
# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
raw_seq=np.array(df_consume_day)
# choose a number of time steps
n_steps = 10
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [14]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)


In [15]:
while True:
    data, addr = sock.recvfrom(1024)
    if(data.decode()=='k'):
        print("Command Received")
        rec_arr=[]
        for i in range(0,10):
            data1, addr1 = sock.recvfrom(1024)
            rec_arr.append(float(data1.decode())/10)
        test_arr=np.array(rec_arr)

        #Demostrate Prediction
        x_input = test_arr
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        predicted_power=yhat[0]*10
        print(predicted_power[0])
        result = client.publish(topic, str(predicted_power[0]),0)

Command Received
67.37916
Command Received
89.950775


KeyboardInterrupt: 